## Calling a parallel simulation code from Julia
### About me
Marc Fuentes, I'm working on simulation software quality at INRIA (French National Institute for Research in Digital Science and Technology) as a research Engineer. Code is available at [Github](http://github.com/aitzkora/OptimizeMPI.jl)

### Rationale
 
We want to do a parameter optimization for a physical process simulated by a parallel software
written in Fortran or C. Thus, implementing a robust optimization method in a low-level language 
could be a waste of time, particularly when you are doing a Ph.D for instance. The aim of this poster
is to share some recipes (technical and numerical), to achieve that in Julia. The knowledge level 
remains deliberately basic, so it can be seen as a tutorial. Our hypotheses are the following : 
- distributed memory paradigm for parallelism (i.e. MPI) 
- direct problem is Fortran or C.
- possibly, implementing a gradient computation will is done in the low level language.
- we focus on continuous optimization (using `Optim.jl`), but principle is the same using JuMP for combinatorial optimization 
As technical points, we want to explain briefly how to call a piece of external code and how to run 
Julia scripts in a MPI environment 

### calling Fortran or C code

To call a external piece of code, we use the *Julia* statement `ccall`. As documented
[here](https://docs.julialang.org/en/v1/base/c/) or [there](https://craftofcoding.wordpress.com/2017/02/08/calling-c-from-julia-i-simple-arrays/), the syntax is the following
```julia
ccall((:funcName, library), returnType, (argType1, argType2, ...), (argVal1, argVal2, ...))
```
where `function_name` is the mangled name of the C function in the shared library library. If you do not know
what is mangling, take a look at [there](https://en.wikipedia.org/wiki/Name_mangling) : Roughly languages like 
Fortran (due to its case insensitivity) or C++ (in which the same function name could have several 
different signatures), must encode their function names when they interoperate with C.

#### Remarks 
- `library` is only _formally_ a string : you could use `"./mylib.so"`, but not `string(pwd(),"/mylib.so")`
- to use a library which is not in `.`, add the path to `LD_LIBRARY_PATH` before launching 
**Julia** (Just tested on Linux, adapt the rule for MacOS with `DYLD_LIBRARY_PATH`)
- using `dlopen` and `dlsym` one could directly use the function pointer call

To start, the following C function `int addTwo(int x) { return x+2; }` could be call
```julia
run(`gcc -o addTwo.so --shared addTwo.c`);
w = ccall((:addTwo, "./addTwo.so"), Int32, (Int32,), 12); println("$w")
```

This example deserves some explanations: 
1. to build a *shared* library we add on the gcc compiler command the flag `--shared`. This is evidently,
compiler dependant. If you use Intel, NAG or Microsoft, it could be different. To enforce more portability,
in provided codes, we use **CMake** as an utility to generate Makefiles and doing the compilation. To
do that, with CMake, one could write `add_library(name_lib SHARED name_src)`

2. One big difference between Fortran and C, is the default argument pass method ; In C, it is by-value,
so a function like `addTwo` cannot modify its arguments. To do that, you need to use pointers and furnish
a Julia **reference** to `ccall`

3. In Fortran, since args are passed by reference, we will use a reference.

```fortran
module example
  use iso_c_binding
contains
  subroutine addTwoF(x) bind(C, name ="addTwoF")
    integer(c_int), intent (inout) :: x
    x = x + 2
  end subroutine 
end module
```

In this example, we used the statement bind to attach a C name to our Fortran function. It will override
the mangled name, when we will use `ccall`. To enforce compatibility, Fortran 90 has some 
C compatibles types , such as  `real(c_double)`, `integer(c_int)` defined in the module `iso_c_binding`.
The `intent(inout)` does not change how the argument is passed (by reference), it is just a information
to enable the compiler to do more checks.

```julia
z = Ref{Int32}(12) # ✏️ VERY IMPORTANT ✏️
w = ccall((:addTwoF, "./addTwoF.so"), Cvoid, (Ref{Int32},), z)
println("z = ",z[])
```

4. To end with external code calling, we have to speak about arrays : Julia arrays can be convert 
to pointers without any problem, when using `ccall` As an example, `void changeArray(int n, double * x) { if (n > 1) x[0] += 3 ; }` could be call by

```julia
a = [1:3.;]; ccall((:changeArray, "./changeArray.so"), Cvoid,(Int32, Ptr{Float64},), size(a,1), a) println("$a")
```

### Interacting with MPI

Interacting with MPI, is not so hard, thanks to the good job done by authors of `MPI.jl` and 
`MPIClusterManagers` packages.a For instance, using `MPI.jl`, one could run 
```julia
using MPI
MPI.Init()
println("Hi from $(MPI.Comm_rank(MPI.COMM_WORLD))!")
flush(stdout)
```
directly from shell `mpirun -np 2 julia examples/hello_world.jl`.

Doing so, since `mpirun` calls julia, the julia code, is JIT-compiled before to execute each time we run 
the script; Furthermore, we must run the code out of the Julia REPL, which is not very  convenient for 
doing some experiments. To avoid that, we can use the `MPIClusterManagers` package's macro `@mpi_do` after
the following preamble
```julia
using MPIClusterManagers, Distributed
manager = MPIManager(np=4)
addprocs(manager)
@everywhere import MPI
```
and running several times the following block without restarting Julia
```julia
@mpi_do manager begin 
    comm = MPI.COMM_WORLD;p = MPI.Comm_size(comm);r = MPI.Comm_rank(comm)
    s_loc=sum(1+r* 100/p:100/p * (r+1)); s=MPI.Allreduce(s_loc, +, comm)
    println("s=$s")
end
```
Unfortunately, the present version of MPIClusterManagers does not have an `@mpi_fetchcall` macro
to retrieve the result of the computation on the master (which is not part of the MPI Cluster).

### Distributed Minimization

#### preliminary remark
Formally, we want to solve a problem like
$$ \min_{x_1,\cdots,x_p} f(x_1,\cdots,x_p) \mbox { where } x_i \in \mathbb{R}^{n_i}$$
the main function $f$ is assumed decomposable, i.e, we could write as a max or a sum of functions defined
on each  $\mathbb{R}^{n_i}$, for instance
$$ f(x_1,\cdots,x_p) = \sum_{i=1}^p f_i(x_i)$$
In almost all optimization methods, you have a **linesearch** step, which try to find the step length 
$\alpha$ to go forward in the descent direction. Since the computation is distributed, each algorithm, 
owns a version of the current step. Fortunately, since the **linesearch** uses only the value of
the function, which it is the same on each process, so $\alpha$ must be the same across processes.

#### A dummy example : the squared $L_2$ norm
Suppose we want to minimize  $f(x) = \frac{1}{2}\|x-c\|^2$ where $c$ is a constant vector. The solution
is trivially equal to $c$. We also remark that if we cut $\mathbb{R}^n$ into some chunks, 
the $f$ function is clearly decomposable and each $f_i$ is just $f_i(x_i) = \frac{1}{2}\|x_i - c_i\|^2$ where
$c_i$ has the chunk components of $c$. Even if this case, we do not need a gradient to find the solution,
one can computes very easily seeing that
$$\nabla f(x) = \bigoplus_{i=1}^p \nabla f_i(x_i) = \bigoplus_{i=1}^p (x_i - c_i) $$.
Since this example is very dumb, we directly build the partition of state vector  $x$ in Julia and computing
$f_i,\nabla f_i$ is coded in Fortran. To partitionate $x$ we computes the chunk sizes according to

```julia
function partition(n::Integer, p::Integer)
 r = n % p
 m = ceil(Integer, n / p)
 part = fill( m, p )
 part[ (r+1) * (r> 0) + (p+1) * (r==0): p ] .= m - 1
 return part
end
```
Then, Julia gather the chunks on the root process
```julia
x_glob = MPI.Gatherv(x_min,Cint[i for i in partition(n,p)] , 0, comm)
```
The low-level part, is simply a computation of the local squared norm and with `MPI_ALLREDUCE`
we shared the global sum among all processes. 
```fortran
subroutine compute_error(n, x, c, f, df) bind(C, name="compute_error")
(...)
  f = 0.d0
  ! computing objective
  f_loc = 0.5d0 * sum( (x - c)**2 )
  call MPI_ALLREDUCE( f_loc, f, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD, ierr )
  ! computing gradient
  df = x - c
end subroutine compute_error
```
Now, we it remains to write the interface using `ccall`
```julia
function simu!(n::Integer, x::Array{Float64,1}, df::Array{Float64,1})
  f = Ref{Float64}(0.)
  c = cos.(1:n)[slice[1]:slice[2]]
  ccall((:compute_error, "./libpar_error.so"), 
  Cvoid, (Ref{Int32}, Ptr{Float64}, Ptr{Float64}, Ref{Float64}, Ptr{Float64}), size(x, 1), x, c, f, df)
  return f[]
end
```
For simplicity, we choose $c_i = \cos(i)$ and the slices are shared as a global variable (very bad!)

We can now choose our favorite minimization algorithm : Most of people likes Nelder-Mead algorithm
because it does not require gradient, but please do not use it,  proofs of convergence towards
non-stationary points exist in the literature [Torczon](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.48.9967) 
 or [McKinnon](https://www.researchgate.net/publication/2763859_Convergence_of_the_Nelder-Mead_simplex_method_to_a_non-stationary_point). Since we have a gradient at our disposal, why not choose a "1.5" order 
method like `LBFGS` ?
 ```julia
res = optimize(cost, grad!, x_loc, LBFGS(), Optim.Options(... ))
 ```
where `cost` and `grad!` are defined by  `cost = x -> simu!(n, x, df_dummy)` and  `grad! = (g,x)-> simu!(n,x,g)`

Note, doing that is costly, because we compute a gradient even if we just need $f$. We will see, in the
next example that we must be able to compute $f$ without computing $\nabla f$.

#### A more advance example : controlling the 2D heat equation

We will now consider a more realistic problem : it remains a toy problem (just 1600 loc), but to solve
it, we will need of a classical tool for time dependent problems : the adjoint method.
Before to explain this point, let us describe the problem

We will heat a square, just applying from the beginning to the end of the simulation the same values 
on the boundary $\Gamma$ of the square. It is our control ; we want obtain for $t=t_{final}$ some values inside 
the square as close as possible to a target function. 
In a continuous world, we want to solve
\begin{eqnarray}
   \min_p && \int_{[0,1]^2} (u(x,t_{final}) - u_{target}(x))^2 \, dx \\
   \mbox { such that }     && \frac{\partial u}{\partial t} (x,t) = \Delta u(x,t) \forall (x,t) \in [0,1]^2 \times [0,t_{final}] \\ 
    && u(x,t) = p(x)  \forall  (x,t) \in \Gamma \times [0,t_{final}]
\end{eqnarray}

We discretize this system, using finite differente scheme for space and Euler formula for time, and obtain
\begin{eqnarray}
   \min_p &&  \|U^n- U^{target}\|^2_{inside}  \\
   \mbox { such that }     && U^{n+1} = F(U^n,p) = U^n - \frac{\delta t}{h^2} \cdot K2D(U^n,p) \\
                           && U^0 = 0. 
\end{eqnarray}

where $K2D$ is the heat kernel with $p$ on the boundary. For those who do not like these formulae,
just remember that we have a recursive sequence of matrices, and controlling boundary matrix 
terms, we want to minimize the error relative to a target matrix at the last iteration. The 
updating scheme is the following

![update scheme](figures/buffer.pdf)

For better explanation, we furnish a Julia sequential version of the problem in the file `heat_seq.jl`
Numericians often call that a 4-stencil. Each interior point of the matrix is computed used its values
and its 4 neighbour values. In the contrary of the previous problem, all the parallel part is done 
in Fortran. The Julia program interacts with the Fortran code, proposing a vector $p$ for the value of the boundary $\Gamma$ and the fortran program could return the value of the function or the gradient : we use 
the `optional` feature of Fortran arguments, and interoperability with C enable us to give null pointer when the argument is not present.
 
##### Adjoint method

Adjoint method are useful, when we have a state equation, for instance, $F(x,p) = 0$ and we want
minimize a function $g(x,p)$ relatively to $p$. Explaining briefly the adjoint method, is not easy,
and we recommend to read the [note of about reccurence](https://github.com/mitmath/18335/blob/master/notes/adjoint/recurrence2.pdf) S. G Johnson. To sum up, to compute a gradient of $g(p)= \| U^n(p) - U^{target} \|^2$, 
we need to apply a direct recursion computing the final state and final error, and after we 
backwardly compute contribution to the gradient using adjoint state. We could another time, take 
the algorithm from `heat_seq.jl` to understand the principle
```julia
function simu(p::Array{Float64,1}, compute_gradient::Bool = true)
 u_f = U_final(p, T) # direct recursion
 v = sum((u_f - vec(target)).^2)
 λ = 2 .*(u_f - vec(target))
 (...) 
 fₚ = ∂ₚF(n) # p-partial derivative ≈ zeroing u in F 
 ∇f = zeros(P)
 for t=T:-dt:dt
     ∇f += fₚ'*λ
     λ = ∂ᵤF(λ) # adjoint update ≈ zeroing p in F
 end
 return v,  ∇f 
end
``` 
Some software tools using *automatic differentiation* exist to generate a program computing the gradient.
For instance, if your code is written in Julia (which is not the purpose of this poster), you could use
[JuliaDiff](https://www.juliadiff.org/). For C or Fortran codes, for instance your could use [adifor](https://www.mcs.anl.gov/research/projects/adifor/) or 
[tapenade](http://tapenade.inria.fr:8080/tapenade/index.jsp). In our case, computing analytical for
the adjoint is very simple. Since $F$ is linear, partial differential of $F$ relative to $U$ and $p$,
is $F$ herself, zeroing the other component.
$$ \partial_uF(u,p)[v] =  v - \frac{\delta t}{h^2} \cdot K2D(v,0) \mbox{ and }
\partial_pF(u,p)[q] =  - \frac{\delta t}{h^2} \cdot K2D(0,q) $$
One could remark that the two operators are very sparse, which is good for us

#### Future works
- In the heat problem, after computing the optimal $p$, we could compute at least the final U.
- benchmarks must be done : to be sure that Fortran parallel code beats Julia sequentiel
- the author wants to write a package in the same spirit as MPI.jl but  for [OpenCoarrays](https://github.com/sourceryinstitute/opencoarrays). MPI is well suited for  parallelism, but it remains a library in Fortran. Coarrays are now part of the standard and they are  more natural to write parallel algorithms.